In [ ]:
from langchain_core.messages import AIMessage
from redbox.models.chain import RedboxQuery, RedboxState, AISettings, configure_agent_task_plan
from uuid import uuid4

state = RedboxState(
    request=RedboxQuery(
        question="Tell me about Some Company and retrieve its interactions",
        # question="what is the sector of the company located in 100 Some Road",
        s3_keys=[],
        user_uuid=uuid4(),
        chat_history=[],
        ai_settings=AISettings(),
        permitted_s3_keys=[],
    )
)
agent_options = {agent.name: agent.name for agent in AISettings().worker_agents}
# agent_options = {"Data_Hub_Agent": "Data_Hub_Agent"}
agent_task, multi_agent_plan = configure_agent_task_plan(agent_options)
tasks = [agent_task(task="Get Some company details", expected_output="company information", agent="Datahub_Agent")]
state.messages = [AIMessage(content=tasks[0].model_dump_json())]

In [ ]:
state.messages

In [ ]:
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from langchain_mcp_adapters.tools import load_mcp_tools
from redbox.graph.agents.configs import agent_configs
from redbox.graph.agents.workers import WorkerAgent
from redbox.models.settings import get_settings

mcp_settings = get_settings().datahub_mcp
mcp_settings.url

In [ ]:
from redbox.models.settings import get_settings

# mcp_settings = get_settings().datahub_mcp
datahub_mcp_url = "http://0.0.0.0:8100/mcp"

async with streamablehttp_client(datahub_mcp_url) as (read, write, _), ClientSession(read, write) as session:
    # Initialize the connection
    await session.initialize()
    # Get tools
    tools = await load_mcp_tools(session)
    # adding URL metadata so that the agent can execute the tool later
    for tool in tools:
        tool.metadata = {"url": datahub_mcp_url}
    # print(await tools[1].ainvoke({"company_name": 'BMW'}))
    # print(asyncio.run(tools[1].ainvoke({"company_name": 'SOME'})))
    config = agent_configs["Datahub_Agent"]
    agent_configs["Datahub_Agent"].tools = tools
    worker = WorkerAgent(config=config)
    response = worker.execute().invoke(state)


response